<a href="https://colab.research.google.com/github/MuzzammilB/EXSK_1111_T03/blob/main/15112024_Github_T04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We have extracted CDS and Exon from mRNA ID.
We found the translation start site position from the CDS and Exon alignment.
We know the exon containing TSS.
We have the exon skip function, which based on the above information skips exons folloeing the exon containing TSS.

What outputs do we want?

(default output)
- Name of the Gene, Exon count, cds length.
- TSS position and exon.
- List of Exons which induces PTC upon skipping.


(checkboxes to select the output)
- Sequence/length of the exons before/after skipping.

In [ ]:
# @title Libraries
# @markdown Installs dependencies
!pip install biopython
!pip install termcolor

from Bio import Entrez, SeqIO
from Bio.Seq import Seq, translate
from termcolor import colored

# Set up your Entrez email
Entrez.email = "muzzammilbhaisaheb@gmail.com"  # replace with your email

In [ ]:
# @title gene
# @markdown provides gene name from mRNA ID.
def gene(mrna_id):
    try:
        # Fetch the mRNA record from NCBI using a context manager
        with Entrez.efetch(db="nucleotide", id=mrna_id, rettype="gb", retmode="text") as handle:
            record = SeqIO.read(handle, "genbank")

        # Look for the gene name in the features
        gene_name = None
        for feature in record.features:
            if feature.type == "gene" and "gene" in feature.qualifiers:
                gene_name = feature.qualifiers["gene"][0]  # Take the first gene name found
                break  # Exit after finding the first gene name

        return gene_name if gene_name else "Gene name not found."

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
# @title get_exons
# @markdown returns exons from mRNA ID.
def get_exons(mrna_id):
    #Fetch mRNA record from NCBI
    try:
        handle = Entrez.efetch(db="nucleotide", id=mrna_id, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "genbank")
        handle.close()

        #Extract exons
        exons = []
        for feature in record.features:
            if feature.type == "exon":
                exons.append(feature.location.extract(record).seq)

        return exons

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [ ]:
# @title get_cds
# @markdown returns coding sequence from mRNA ID
def get_cds(mrna_id):
    try:
        # Fetch the GenBank record for the given mRNA ID
        handle = Entrez.efetch(db="nucleotide", id=mrna_id, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "genbank")
        handle.close()

        # Search for the CDS feature in the GenBank record
        for feature in record.features:
            if feature.type == "CDS":
                # Extract the CDS sequence
                cds_seq = feature.extract(record.seq)
                cds_product = feature.qualifiers.get('product', ['Unknown Product'])[0]

                return cds_seq

        # If no CDS is found
        print(colored(f"No CDS annotation found for NCBI mRNA ID: {mrna_id}", "red"))
        return None
    except Exception as e:
        print(colored(f"Error: {str(e)}", "red"))
        return None

In [ ]:
# @title find_tss
# @markdown returns TSS (translation start site) position by aligning sequences of mRNA with CDS.
def find_tss(mrna_id):

    exons = get_exons(mrna_id)
    if not exons:
        print("No exons found.")
        return None

    mrna_seq = "".join([str(exon) for exon in exons])

    cds_seq = get_cds(mrna_id)
    if cds_seq is None:
        print("No CDS sequence found.")
        return None

    start_position = mrna_seq.find(str(cds_seq))

    if start_position == -1:
        print("CDS not found within the mRNA sequence.")
        return None

    return start_position + 1

In [ ]:
# @title exon_with_tss
# @markdown returns the index of the exon containing TSS
# always put start_position = find_tss

def exon_with_tss(exons, start_position):
    cumulative_length = 0
    for i, exon in enumerate(exons):
        cumulative_length += len(exon)
        if cumulative_length >= start_position:
            return i

    #The TSS is located in exon {exon_with_tss + 1}
    return None

In [ ]:
# @title ESK_CDSv03
# @markdown simulates exon skipping and translates modified mRNA starting from the next exon containing TSS

def exon_skipping(mrna_id):

    start_position = find_tss(mrna_id)

    if start_position is None:
        print("Unable to find start position for CDS.")
        return None

    exons = get_exons(mrna_id)
    if not exons:
        print("No exons found.")
        return None

    start_exon_index = exon_with_tss(exons, start_position)

    if start_exon_index is None:
        print("Unable to find the exon containing the start position.")
        return None

    mrna_seq = "".join([str(exon) for exon in exons])
    normal_protein_allptc = translate(mrna_seq[start_position - 1:])
    normal_protein_1stptc = translate(mrna_seq[start_position - 1:], to_stop=True)
    normal_diff = len(normal_protein_allptc) - len(normal_protein_1stptc)
    #print(f"Normal Protein length_allptc: {len(normal_protein_allptc)}")
    print(f"Protein length: {len(normal_protein_1stptc)} aa")

    # skipping exons starting from the exon next to the exon containing the start codon
    for i in range(start_exon_index + 1, len(exons)):
        # Create a modified mRNA sequence by skipping the i-th exon onwards
        modified_exons = exons[:i] + exons[i+1:]  # Skip the i-th exon
        modified_mrna_seq = "".join([str(exon) for exon in modified_exons])

        # Ensure that we only translate if the start position is within bounds
        if start_position - 1 < len(modified_mrna_seq):
            # Translate the modified mRNA sequence
            mod_protein_allptc = translate(modified_mrna_seq[start_position - 1:])
            mod_protein_1stptc = translate(modified_mrna_seq[start_position - 1:], to_stop=True)  # Adjust for 0-based index

            if (len(mod_protein_allptc) - len(mod_protein_1stptc)) > normal_diff:
                print(f"\nSkipping exon {i + 1}:")
                print(f"Modified mRNA length: {len(modified_mrna_seq)}")
                print(f"Modified mRNA Sequence: {modified_mrna_seq}")
                print(f"Translated Protein Sequence_allptc: {mod_protein_allptc}")
                #print(f"Translated Protein length_allptc: {len(mod_protein_allptc)}")
                #print(f"Translated Protein Sequence_1stptc: {mod_protein_1stptc}")
                print(f"Translated Protein length: {len(mod_protein_1stptc)}")

        else:
            print(f"Start position {start_position} is out of bounds after skipping exon {i + 1}.")

In [ ]:
# @title Github Input-output
#mrna_id = "NM_002046.7"  #GAPDH
mrna_id = "NM_000492.4"  #CFTR
#mrna_id = "NM_170707.4"  #LMNA
#mrna_id = "NM_001165963.4"  #SCN1A
#mrna_id = "NM_000518.5" #Heamoglobin



exons = get_exons(mrna_id)  # List of exon sequences
print(f"GENE: {gene(mrna_id)}")
print(f"Exons: {len(get_exons(mrna_id))}")
print((f"CDS Length: {len(get_cds(mrna_id))} nt"))
print(f"TSS: {find_tss(mrna_id)} nt")
print(f"TSS is in exon: {exon_with_tss(exons, (find_tss(mrna_id))) + 1}")


exon_skipping(mrna_id)